In [1]:
import torch
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [2]:
model_path = 'georgian_spellcheck_seq2seq.pt'
checkpoint = torch.load(model_path, map_location = device)

itos = checkpoint['vocab']['itos']
stoi = checkpoint['vocab']['stoi']
pad_idx = checkpoint['vocab']['pad_idx']
sos_idx = checkpoint['vocab']['sos_idx']
eos_idx = checkpoint['vocab']['eos_idx']
unk_idx = checkpoint['vocab']['unk_idx']

embedding_dim = checkpoint['config']['embedding_dim']
hidden_dim = checkpoint['config']['hidden_dim']
num_layers = checkpoint['config']['num_layers']
dropout = checkpoint['config']['dropout']
max_target_len = checkpoint['config']['max_target_len']

vocab_size = len(itos)

In [3]:
def encode_word(word, add_eos = True):
    x = []
    for ch in word:
        x.append(stoi.get(ch, unk_idx))
    if add_eos:
        x.append(eos_idx)
    return x

def decode_indices(indices):
    chars = []
    for idx in indices:
        if idx == eos_idx or idx == pad_idx:
            break
        chars.append(itos[idx])
    return ''.join(chars)

In [4]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers = 1, dropout = 0.1):
        super().__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings = vocab_size,
            embedding_dim = embedding_dim,
            padding_idx = pad_idx
        )

        self.gru = nn.GRU(
            input_size = embedding_dim,
            hidden_size = hidden_dim,
            num_layers = num_layers,
            batch_first = True,
            dropout = dropout if num_layers > 1 else 0.0,
            bidirectional = False
        )

    def forward(self, src, src_lengths):
        embedded = self.embedding(src)

        packed = nn.utils.rnn.pack_padded_sequence(
            embedded,
            src_lengths.cpu(),
            batch_first = True,
            enforce_sorted = False
        )

        outputs, hidden = self.gru(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first = True)
        return outputs, hidden

In [5]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim

    def forward(self, hidden, encoder_outputs, src_lengths):
        batch, src_len, hidden_dim = encoder_outputs.size()

        query = hidden[-1]

        scores = torch.bmm(
            encoder_outputs,
            query.unsqueeze(2),
        ).squeeze(2)

        device_scores = scores.device
        mask = torch.arange(src_len, device = device_scores).unsqueeze(0) >= src_lengths.unsqueeze(1)
        scores = scores.masked_fill(mask, float('-inf'))

        attn_weights = torch.softmax(scores, dim = 1)

        context = torch.bmm(
            attn_weights.unsqueeze(1),
            encoder_outputs,
        ).squeeze(1)

        return context, attn_weights

In [6]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers = 1, dropout = 0.1):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings = vocab_size,
            embedding_dim = embedding_dim,
            padding_idx = pad_idx
        )

        self.attention = Attention(hidden_dim)

        self.gru = nn.GRU(
            input_size = embedding_dim + hidden_dim,
            hidden_size = hidden_dim,
            num_layers = num_layers,
            batch_first = True,
            dropout = dropout if num_layers > 1 else 0.0
        )

        self.fc_out = nn.Linear(hidden_dim * 2, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_step, hidden, encoder_outputs, src_lengths):
        embedded = self.dropout(self.embedding(input_step))
        embedded = embedded.unsqueeze(1)

        context, _ = self.attention(hidden, encoder_outputs, src_lengths)
        context = context.unsqueeze(1)

        rnn_input = torch.cat([embedded, context], dim = 2)

        output, hidden = self.gru(rnn_input, hidden)
        output = output.squeeze(1)
        context = context.squeeze(1)
        concat = torch.cat([output, context], dim = 1)
        logits = self.fc_out(concat)

        return logits, hidden

In [7]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, sos_idx, eos_idx, pad_idx, max_target_len):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        self.max_target_len = max_target_len

    def forward(self, src, src_lengths, tgt = None, teacher_forcing_ratio = 0.0):
        batch_size = src.size(0)
        encoder_outputs, hidden = self.encoder(src, src_lengths)
        device_local = src.device

        if tgt is not None:
            max_len = tgt.size(1)
        else:
            max_len = self.max_target_len

        outputs = torch.zeros(batch_size, max_len, vocab_size, device = device_local)

        input_step = torch.full(
            (batch_size,),
            self.sos_idx,
            dtype = torch.long,
            device = device_local
        )

        for t in range(max_len):
            logits, hidden = self.decoder(
                input_step,
                hidden,
                encoder_outputs,
                src_lengths
            )

            outputs[:, t, :] = logits
            input_step = logits.argmax(dim = 1)

        return outputs

In [8]:
encoder = Encoder(vocab_size, embedding_dim, hidden_dim, num_layers, dropout)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim, num_layers, dropout)

model = Seq2Seq(
    encoder,
    decoder,
    sos_idx,
    eos_idx,
    pad_idx,
    max_target_len
).to(device)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print('Model loaded')


Model loaded


In [9]:
def correct_word(word, model = model):
    src_indices = encode_word(word)
    src_tensor = torch.tensor(src_indices, dtype = torch.long).unsqueeze(0).to(device)
    src_length = torch.tensor([len(src_indices)], dtype = torch.long).to(device)

    with torch.no_grad():
        logits = model(src_tensor, src_length)

    token_indices = logits.argmax(dim = 2)[0].tolist()
    return decode_indices(token_indices)

In [10]:
with open('georgian_words.txt', 'r', encoding = 'utf-8') as f:
    all_words_list = [w for w in f.read().split(',') if w]

print("Loaded words:", len(all_words_list))

Loaded words: 23162


In [11]:
import random

georgian_keyboard_map = [
    ['ქ', 'წჭ', 'ე', 'რღ', 'ტთ', 'ყ', 'უ', 'ი', 'ო', 'პ'],
    ['ა', 'სშ', 'დ', 'ფ', 'გ', 'ჰ', 'ჯჟ', 'კ', 'ლ', None],
    ['ზძ', 'ხ', 'ცჩ', 'ვ', 'ბ', 'ნ', 'მ', None, None, None],
]

n, m = len(georgian_keyboard_map), len(georgian_keyboard_map[0])

dirs = [
    (-1, -1), (-1, 0), (-1, 1),
    ( 0, -1), ( 0, 0), ( 0, 1),
    ( 1, -1), ( 1, 0), ( 1, 1),
]

step_probabilities = [
    0.00005, 0.00020, 0.00005,
    0.00020, 0.99900, 0.00020,
    0.00005, 0.00200, 0.00005,
]

def keyboard_typo(word, shift_change_prob = 0.05):
    char_to_pos = {}
    for i, row in enumerate(georgian_keyboard_map):
        for j, cell in enumerate(row):
            if cell is not None:
                for shift_idx, ch in enumerate(cell):
                    char_to_pos[ch] = (i, j, shift_idx)
    out = []
    for ch in word:
        if ch not in char_to_pos:
            out.append(ch)
            continue
        row, col, shift_idx = char_to_pos[ch]
        idx = random.choices(range(len(dirs)), weights = step_probabilities)[0]
        dr, dc = dirs[idx]
        nr, nc = row + dr, col + dc
        if 0 <= nr < n and 0 <= nc < m:
            target_cell = georgian_keyboard_map[nr][nc]
            if target_cell is None:
                continue
            if random.random() < shift_change_prob and len(target_cell) > 1:
                ns = 1 - shift_idx if shift_idx < 2 else 0
                ns = min(ns, len(target_cell) - 1)
            else:
                ns = min(shift_idx, len(target_cell) - 1)
            out.append(target_cell[ns])
        else:
            out.append(ch)
    return ''.join(out)

def swap_adjacent_chars(word, swap_prob = 0.005):
    chars = list(word)
    p = swap_prob
    i = 0
    while i < len(chars) - 1:
        if random.random() < p:
            chars[i], chars[i + 1] = chars[i + 1], chars[i]
            p /= 10
            i += 2
        else:
            i += 1
    return ''.join(chars)

def double_char(word, double_prob = 0.005):
    out = []
    p = double_prob
    for ch in word:
        out.append(ch)
        if random.random() < p:
            out.append(ch)
            p /= 10
    return ''.join(out)

def corrupt_word(word):
    w = keyboard_typo(word)
    w = swap_adjacent_chars(w)
    w = double_char(w)
    return w

In [12]:
random.seed(42)

sample_words = random.sample(all_words_list, 1000)

results = []

def edit_distance(a, b):
    n, m = len(a), len(b)
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    for i in range(n + 1): dp[i][0] = i
    for j in range(m + 1): dp[0][j] = j
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if a[i - 1] == b[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])
    return dp[n][m]

for word in sample_words:
    corrupted = corrupt_word(word)
    corrected = correct_word(corrupted)
    results.append((word, corrupted, corrected))


In [ ]:
char_total = 0
char_correct = 0

word_total = 0
word_correct = 0

edit_sum = 0

for gold, corrupted, predicted in results:
    for c1, c2 in zip(gold, predicted):
        char_total += 1
        if c1 == c2:
            char_correct += 1

    word_total += 1
    if gold == predicted:
        word_correct += 1

    edit_sum += edit_distance(gold, predicted)

char_acc = char_correct / max(char_total, 1)
word_acc = word_correct / word_total
edit_avg = edit_sum / word_total

print('Character accuracy:', char_acc)
print('Word accuracy:', word_acc)
print('Average edit distance:', edit_avg)


Character accuracy: 0.9754837158995313
Word accuracy: 0.913
Average edit distance: 0.14


In [ ]:
print('აალაპარაკდ' in all_words_list)

True


In [40]:
correct_cnt = 0
print('Correctly fixed words:\n')

for real_world, corrupted, predicted in results:
    if corrupted != real_world and predicted == real_world:
        correct_cnt += 1
        print(corrupted, "->", predicted, "| real_world:", real_world)

print()
print('Fixed a total of', correct_cnt, 'words')

Correctly fixed words:

ეედავება -> ედავება | real_world: ედავება
გააფთღებით -> გააფთრებით | real_world: გააფთრებით
შშეუხრია -> შეუხრია | real_world: შეუხრია
იჭრთ -> ჭირთ | real_world: ჭირთ
მოსამშახურეთზ -> მოსამსახურეთა | real_world: მოსამსახურეთა
გლეეხის -> გლეხის | real_world: გლეხის
ტვალიანი -> თვალიანი | real_world: თვალიანი
ფხიძლად -> ფხიზლად | real_world: ფხიზლად
წყალთაშა -> წყალთასა | real_world: წყალთასა
ზღარტანი -> ზღართანი | real_world: ზღართანი
განმსაძღვრელ -> განმსაზღვრელ | real_world: განმსაზღვრელ
წყოობილებას -> წყობილებას | real_world: წყობილებას
ჩატვლით -> ჩათვლით | real_world: ჩათვლით
ნივთშ -> ნივთს | real_world: ნივთს
სასჯელიშ -> სასჯელის | real_world: სასჯელის
ეშარიგა -> შეარიგა | real_world: შეარიგა
მოუნდესს -> მოუნდეს | real_world: მოუნდეს
გასაზფდელად -> გასაზრდელად | real_world: გასაზრდელად
ემასლაატებოდა -> ემასლაათებოდა | real_world: ემასლაათებოდა
ბოსნიუღი -> ბოსნიური | real_world: ბოსნიური
არათანნმიმდევრულად -> არათანმიმდევრულად | real_world: არათანმიმდევრულად
მ

In [41]:
incorrect_cnt = 0
print('Not fixed words:\n')

for real_world, corrupted, predicted in results:
    if corrupted != real_world and predicted != real_world:
        incorrect_cnt += 1
        print(corrupted, "->", predicted, "| real_world:", real_world)

print('Couldn\'t fix a total of', incorrect_cnt, 'words')


Not fixed words:

გამოიკვთეოს -> გამოიკვთეოს | real_world: გამოიკვეთოს
ბავყოფთ -> ბავყოფთ | real_world: გავყოფთ
ააღიანი -> არიანი | real_world: აღაიანი
ეხებდოა -> ეხებდოა | real_world: ეხებოდა
მოენდისკენ -> მოენდისკენ | real_world: მოედნისკენ
შინაარწობრივ -> სინაარწორივ | real_world: შინაარსობრივ
ელიოტკ -> ელიოოტ | real_world: ელიოტი
მოღვწაეობამ -> მოღვწარებოაა | real_world: მოღვაწეობამ
საწთუმალი -> საწთუმალი | real_world: სასთუმალი
აღვნშინავთ -> აღვნშინავთ | real_world: აღვნიშნავთ
დანბელების -> დანბელების | real_world: დაბნელების
ნაბიჭვრაო -> ნაბიჭვრაო | real_world: ნაბიჭვარო
სქვალდებულოა -> სქვალდებულოა | real_world: სავალდებულოა
სააზუღგეს -> სააზურგეს | real_world: საზურგეს
ორლი -> ორლი | real_world: როლი
ახლომხდეველი -> ახლომხდეველი | real_world: ახლომხედველი
თოდირა -> თოდირა | real_world: თოდრია
ბრიტანდთმა -> ბრიტანდთმა | real_world: ბრიტანეთმა
ჰერონიი -> ჰერონი | real_world: ჰეროინი
თთველმდნენ -> თველმდნენ | real_world: თვლემდნენ
ზაავთებული -> ზავთებული | real_world: აზავთებული
გ